# libraries

In [75]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import time
import scipy.optimize as optimize
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from scipy.stats import rankdata

from collections import defaultdict
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import TransformerMixin, BaseEstimator
from tqdm import tqdm
import joblib

import time
import re 
import scipy
from scipy import sparse
import gc 
from IPython.display import display, HTML
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")

# set

In [76]:
#1 means kf 2 means sample
Fold_type=2

data_names=["jc_"]
model_choice=["ridge"]
factor=[1]

use_new_val=False

# Data

In [77]:
system_path=r"C:\Users\Lenovo\Desktop\stupidcode\data\jigsaw"
#第一届 jigsaw比赛 数据（challenge） Toxic Comment Classification Challenge

jc_path=os.path.join(system_path,"jigsaw-toxic-comment-classification-challenge")
#ruddit 数据
run_path=os.path.join(system_path,"ruddit-jigsaw-dataset/Dataset")
#第二届 jigsaw比赛 对少数人群不歧视
juc_path=os.path.join(system_path,"jigsaw-unintended-bias-in-toxicity-classification")

#本次比赛数据 作为val
jts_path=os.path.join(system_path,"jigsaw-toxic-severity-rating")

# #数据抽样存储路径

gbm_save_path=os.path.join(system_path,r"save_model\four_model ridge+gbm 0.806")
ridge_save_path=os.path.join(system_path,r"save_model\four_model ridge+gbm 0.806")



In [78]:
#验证集和测试集
df_val = pd.read_csv(os.path.join(jts_path,"validation_data.csv"))

df_test = pd.read_csv(os.path.join(jts_path,"comments_to_score.csv"))

In [79]:
# 第一届比赛数据 以0/1为分值 
features = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

jc_train_df = pd.read_csv(os.path.join(jc_path,"train.csv"))
jc_test_df = pd.read_csv(os.path.join(jc_path,"test.csv"))
temp_df = pd.read_csv(os.path.join(jc_path,"test_labels.csv"))

jc_test_df = jc_test_df.merge ( temp_df, on ="id")
#drop test data not used for scoring
jc_test_df = jc_test_df.query ("toxic != -1")
jc_df = jc_train_df.append ( jc_test_df ) 

print(f"Train+Test:{jc_df.shape[0]}")

# 将代表有毒行为的筛选出来
jc_df["toxic_subtype_sum"]=jc_df[features].sum(axis=1)
jc_df["toxic_behaviour"]=jc_df["toxic_subtype_sum"].map(lambda x: x > 0)

tot_toxic_behaviour = jc_df["toxic_behaviour"].sum()
print(f'comments with toxic behaviour:{tot_toxic_behaviour}')

jc_df = jc_df[['comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].rename(columns={'comment_text': 'text'})

Train+Test:223549
comments with toxic behaviour:22468


In [80]:
# 消除jc中重叠数据
df_val_new = pd.read_csv(os.path.join(jts_path,"validation_data.csv"))
print(df_val_new.shape)


# Find cases already present in toxic data

df_val_new = pd.merge(df_val_new, jc_df.loc[:,['text']], 
                  left_on = 'less_toxic', 
                  right_on = 'text', how='left')

df_val_new = pd.merge(df_val_new, jc_df.loc[:,['text']], 
                  left_on = 'more_toxic', 
                  right_on = 'text', how='left')

# Removing those cases
df_val_new = df_val_new[(df_val_new.text_x.isna()) & (df_val_new.text_y.isna())][['worker', 'less_toxic', 'more_toxic']]
df_val_new.shape

(30108, 3)


(6489, 3)

In [81]:
if use_new_val==True:
    df_val=df_val_new

In [82]:
Fold_type=2
if Fold_type==1:
    fold_num=5
else:
    fold_num=3

In [83]:
if Fold_type==1:
    pre_names=[ data_name+"k_" for data_name in data_names]
elif Fold_type==2:
    pre_names=[ data_name+"s_" for data_name in data_names]

In [84]:
#clean data
def clean(data, col):
    #数据清洗 
    
    # Clean some punctutations
    data[col] = data[col].str.replace('\n', ' \n ')
    data[col] = data[col].str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3')
    # Replace repeating characters more than 3 times to length of 3
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}',r'\1\1\1')    
    # Add space around repeating characters
    data[col] = data[col].str.replace(r'([*!?\']+)',r' \1 ')    
    # patterns with repeating characters 
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b',r'\1\1')
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1')
    data[col] = data[col].str.replace(r'[ ]{2,}',' ').str.strip()   
    
    return data

In [85]:
def ridge_cv(model_pre,clean_prm=False):
    val_preds_arr1 = np.zeros((df_val.shape[0], fold_num))
    val_preds_arr2 = np.zeros((df_val.shape[0], fold_num))
    test_preds_arr = np.zeros((df_test.shape[0], fold_num))
    for fld in range(fold_num):
        model_path=os.path.join(ridge_save_path,f"{model_pre}{fld}.pkl")
        vec_model_path=os.path.join(ridge_save_path,f"{model_pre}vec_{fld}.pkl")
        
        vec = joblib.load(vec_model_path)
        model=joblib.load(model_path)
        
        if clean_prm==False:
            X_less_toxic = vec.transform(clean(df_val,'less_toxic')['less_toxic'])
            X_more_toxic = vec.transform(clean(df_val,'more_toxic')['more_toxic'])
            X_test = vec.transform(clean(df_test,'text')['text'])

        else:
            X_less_toxic = vec.transform(df_val['less_toxic'])
            X_more_toxic = vec.transform(df_val['more_toxic'])
            X_test=vec.transform(df_test['text'])
            
        val_preds_arr1[:,fld] = model.predict(X_less_toxic)
        val_preds_arr2[:,fld] = model.predict(X_more_toxic)
        test_preds_arr[:,fld] = model.predict(X_test)
        
        del model,vec
    p1=val_preds_arr1.mean(axis=1)
    p2=val_preds_arr2.mean(axis=1)
    pv=test_preds_arr.mean(axis=1)
    print(f'Ridge Validation Accuracy is { np.round((p1 < p2).mean() * 100,2)}')   
    return p1,p2,pv

In [86]:
def lightgbm_cv(model_pre,clean_prm=False):
    val_preds_arr1 = np.zeros((df_val.shape[0], fold_num))
    val_preds_arr2 = np.zeros((df_val.shape[0], fold_num))
    test_preds_arr = np.zeros((df_test.shape[0], fold_num))
    
    for fld in range(fold_num):
        
        model_path=os.path.join(gbm_save_path,f"{model_pre}{fld}.txt")
        vec_model_path=os.path.join(gbm_save_path,f"{model_pre}vec_{fld}.pkl")
        # 模型加载
        vec = joblib.load(vec_model_path)
        gbm = lgb.Booster(model_file=model_path)
        if clean_prm==False:
            X_less_toxic = vec.transform(clean(df_val,'less_toxic')['less_toxic'])
            X_more_toxic = vec.transform(clean(df_val,'more_toxic')['more_toxic'])
            X_test = vec.transform(clean(df_test,'text')['text'])
            
        else:
            X_less_toxic = vec.transform(df_val['less_toxic'])
            X_more_toxic = vec.transform(df_val['more_toxic'])
            X_test=vec.transform(df_test['text'])
            
        val_preds_arr1[:,fld] = gbm.predict(X_less_toxic,num_iteration=gbm.best_iteration)
        val_preds_arr2[:,fld] = gbm.predict(X_more_toxic,num_iteration=gbm.best_iteration)
        test_preds_arr[:,fld] = gbm.predict(X_test, num_iteration=gbm.best_iteration)
        del gbm,vec
        
    p1=val_preds_arr1.mean(axis=1)
    p2=val_preds_arr2.mean(axis=1)
    pv=test_preds_arr.mean(axis=1)
    print(f'Gbm Validation Accuracy is { np.round((p1 < p2).mean() * 100,2)}')        
    return p1,p2,pv

In [87]:
p1=defaultdict()
p2=defaultdict()
pv=defaultdict()

func_dict={"ridge":ridge_cv,"gbm":lightgbm_cv}
score=np.zeros(df_test.shape[0])

p1_ensenmble = np.zeros((df_val.shape[0]))
p2_ensenmble = np.zeros((df_val.shape[0]))

for pre_name in tqdm(pre_names):
    ###model_pre_ridge:jc_k_ridge_ pre_name:jc_k_ model:jc_k_ridge_{fold} vec:jc_k_ridge_vec_{fold}
    #pre_name ju_k_ model_name jc_k_ridge_
    clean_prm=False
    p1[pre_name],p2[pre_name]=np.zeros((df_val.shape[0])),np.zeros((df_val.shape[0]))
    pv[pre_name]=np.zeros((df_test.shape[0]))
    if "jcc" in pre_name:
        clean_prm=True
    for index,model_name in enumerate(model_choice):
        cv_func=func_dict.get(model_name)
        model_pre=pre_name+model_name+"_"
        p1[model_pre],p2[model_pre],pv[model_pre]=cv_func(model_pre,clean_prm)
        
        p1[pre_name]= p1[pre_name]+ p1[model_pre]*factor[index]
        p2[pre_name]= p2[pre_name]+ p2[model_pre]*factor[index]
        pv[pre_name]= pv[pre_name]+ pv[model_pre]*factor[index]

    kmax=max(p1[pre_name].max(),p2[pre_name].max())
    p1_ensenmble=p1_ensenmble+p1[pre_name]/kmax
    p2_ensenmble=p2_ensenmble+p2[pre_name]/kmax
    score=score+pv[pre_name]/kmax
    
print(f' Validation Accuracy is { np.round((p1_ensenmble < p2_ensenmble).mean() * 100,2)}') 

df_test['score'] = rankdata(score, method='ordinal')

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [01:57<00:00, 117.17s/it]

Gbm Validation Accuracy is 67.62
k Validation Accuracy is 67.62


In [88]:
df_test[['comment_id', 'score']].to_csv("submission.csv", index=False)
